In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("train.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
df=df.fillna('')

In [8]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [10]:
df=df.drop(['id', 'title', 'author'], axis=1)

In [11]:
df.head()

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
from nltk.corpus import stopwords

In [13]:
from nltk.stem.porter import PorterStemmer

In [14]:
import re

In [15]:
port_stem=PorterStemmer()

In [16]:
port_stem

<PorterStemmer>

In [17]:
port_stem.stem("Hi thIs is chando * % %@@@")

'hi this is chando * % %@@@'

In [18]:
def stemming(content):
    con=re.sub('[^a-zA-Z]', ' ', content)
    con=con.lower()
    con=con.split()
    con=[port_stem.stem(word) for word in con if not word in stopwords.words('english')]
    con=' '.join(con)
    return con

In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
stemming('Hi this is chando')

'hi chando'

In [22]:
df['text']= df['text'].apply(stemming)

In [24]:
x=df['text']

In [25]:
y=df['label']

In [26]:
y.shape

(20800,)

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
x_train , x_test , y_train, y_test = train_test_split(x, y, test_size=0.20)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
vect=TfidfVectorizer()

In [31]:
x_train=vect.fit_transform(x_train)
x_test=vect.transform(x_test)

In [32]:
x_test.shape

(4160, 97673)

In [33]:
from sklearn.tree import DecisionTreeClassifier

In [34]:
model=DecisionTreeClassifier()

In [35]:
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [36]:
prediction=model.predict(x_test)

In [37]:
prediction

array([0, 1, 0, ..., 0, 0, 0])

In [38]:
model.score(x_test, y_test)

0.8810096153846154

In [39]:
import pickle

In [40]:
pickle.dump(vect, open('vector.pkl', 'wb'))

In [41]:
pickle.dump(model, open('model.pkl', 'wb'))

In [42]:
vector_form=pickle.load(open('vector.pkl', 'rb'))

In [43]:
load_model=pickle.load(open('model.pkl', 'rb'))

In [44]:
def fake_news(news):
    news=stemming(news)
    input_data=[news]
    vector_form1=vector_form.transform(input_data)
    prediction = load_model.predict(vector_form1)
    return prediction

In [45]:
val=fake_news("""In these trying times, Jackie Mason is the Voice of Reason. [In this week’s exclusive clip for Breitbart News, Jackie discusses the looming threat of North Korea, and explains how President Donald Trump could win the support of the Hollywood left if the U. S. needs to strike first.  “If he decides to bomb them, the whole country will be behind him, because everybody will realize he had no choice and that was the only thing to do,” Jackie says. “Except the Hollywood left. They’ll get nauseous. ” “[Trump] could win the left over, they’ll fall in love with him in a minute. If he bombed them for a better reason,” Jackie explains. “Like if they have no transgender toilets. ” Jackie also says it’s no surprise that Hollywood celebrities didn’t support Trump’s strike on a Syrian airfield this month. “They were infuriated,” he says. “Because it might only save lives. That doesn’t mean anything to them. If it only saved the environment, or climate change! They’d be the happiest people in the world. ” Still, Jackie says he’s got nothing against Hollywood celebs. They’ve got a tough life in this country. Watch Jackie’s latest clip above.   Follow Daniel Nussbaum on Twitter: @dznussbaum """)

In [46]:
if val==[0]:
    print('reliable')
else:
    print('unreliable')

reliable
